# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import Ridge
from matplotlib.colors import Normalize
import networkx as nx
import itertools
from tqdm import tqdm
from scipy.signal import welch
import matplotlib.cm as cm
import seaborn as sns
from collections import defaultdict
from sklearn.decomposition import PCA
import json

# Baselines

In [2]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

# MPPR

In [3]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    """Numerically stable logistic function."""
    return 1.0 / (1.0 + np.exp(-x))


class MPPRN:
    """
    Swirl-Gated k-Cycle Echo-State Network (SG-kC-ESN).

    Parameters
    ----------
    reservoir_size : int
        Total number of neurons N (must be divisible by n_cycles).
    n_cycles       : int, ≥ 2
        How many simple cycles to create.
    cycle_weight   : float
        Weight r on each ring edge.
    bridge_weight  : float
        Weight s on the k sparse bridges (one per cycle).
    input_scale    : float
        Scaling of random input matrix W_in.
    leak_rate      : float in (0,1]
        leaky-integrator update; 1 recovers standard ESN.
    ridge_alpha    : float
        ℓ₂ penalty used in the ridge read-out.
    swirl_beta, swirl_frequency, swirl_sigmoid
        Parameters of the static per-neuron swirl gate
            g_k = σ[β sin(ω·q + φ_c)]      if swirl_sigmoid
                  β sin(ω·q + φ_c)         otherwise
        with φ_c = 2πc / k   (c = ring id).
    """

    # ------------------------------- init ------------------------------ #
    def __init__(
        self,
        reservoir_size: int = 600,
        n_cycles: int = 4,
        cycle_weight: float = 0.9,
        bridge_weight: float = 0.25,
        input_scale: float = 0.5,
        leak_rate: float = 1.0,
        ridge_alpha: float = 1e-6,
        swirl_beta: float = 2.0,
        swirl_frequency: float | None = None,
        swirl_sigmoid: bool = True,
        seed: int = 42,
        use_polynomial_readout: bool = True,
    ):
        if n_cycles < 2:
            raise ValueError("n_cycles must be at least 2")
        if reservoir_size % n_cycles:
            raise ValueError("reservoir_size must be divisible by n_cycles")

        # -------------- basic bookkeeping --------------------------------
        self.N = reservoir_size
        self.k = n_cycles
        self.m = reservoir_size // n_cycles          # neurons per ring

        # -------------- hyper-parameters ---------------------------------
        self.r = cycle_weight
        self.s = bridge_weight
        self.input_scale = input_scale
        self.alpha = leak_rate
        self.ridge_alpha = ridge_alpha
        self.beta = swirl_beta
        self.omega = (
            swirl_frequency if swirl_frequency is not None else 2.0 * np.pi / self.m
        )
        self.swirl_sigmoid = swirl_sigmoid
        self.seed = seed
        self.use_poly = use_polynomial_readout

        # -------------- placeholders to be filled ------------------------
        self.W_res: np.ndarray | None = None
        self.W_in: np.ndarray | None = None
        self.W_out: np.ndarray | None = None
        self.gate: np.ndarray | None = None
        self.x = np.zeros(self.N, dtype=np.float32)

        # -------------- one-off construction -----------------------------
        self._build_reservoir()
        self._build_swirl_gate()

    # =========================== builders =============================== #
    def _build_reservoir(self):
        """Construct the k-cycle recurrent matrix W_res (shape N × N)."""
        m, r, s, k = self.m, self.r, self.s, self.k

        # 1) ring block C_r : unidirectional permutation matrix scaled by r
        C_r = np.zeros((m, m), dtype=np.float32)
        for i in range(m):
            C_r[(i + 1) % m, i] = r

        # 2) bridge block S : rank-1 matrix with single non-zero entry (0,0)
        S = np.zeros((m, m), dtype=np.float32)
        S[0, 0] = s

        # 3) assemble full block matrix
        W = np.zeros((self.N, self.N), dtype=np.float32)

        def put_block(row_ring: int, col_ring: int, block: np.ndarray):
            i0, j0 = row_ring * m, col_ring * m
            W[i0 : i0 + m, j0 : j0 + m] = block

        for c in range(k):
            put_block(c, c, C_r)                   # diagonal ring
            put_block(c, (c - 1) % k, S)           # bridge to predecessor

        self.W_res = W

    def _build_swirl_gate(self):
        """Pre-compute static gain vector g (length N)."""
        g = np.empty(self.N, dtype=np.float32)
        for k_idx in range(self.N):
            ring_id = k_idx // self.m
            local_q = k_idx % self.m
            phi_c = 2.0 * np.pi * ring_id / self.k
            raw = self.beta * np.sin(self.omega * local_q + phi_c)
            g[k_idx] = sigmoid(raw) if self.swirl_sigmoid else raw
        self.gate = g

    # ====================== low-level reservoir ops ===================== #
    def _apply_gate(self, vec: np.ndarray) -> np.ndarray:
        return self.gate * vec

    def _update_state(self, u_t: np.ndarray):
        """Single ESN step with optional leakage."""
        pre = self.W_res @ self.x + self.W_in @ u_t
        gated = self._apply_gate(pre)
        new_x = np.tanh(gated)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * new_x

    def reset_state(self):
        self.x.fill(0.0)

    # ====================== read-out training (ridge) =================== #
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher forcing pass • inputs [T, d_in] → states, then fit ridge.
        """
        T, d_in = inputs.shape
        if T <= discard + 1:
            raise ValueError("Not enough data for training")

        rng = np.random.default_rng(self.seed)
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.N, d_in)) * self.input_scale
        ).astype(np.float32)

        self.reset_state()
        states = []
        for t in range(T):
            self._update_state(inputs[t])
            if t >= discard:
                states.append(self.x.copy())

        states = np.asarray(states, dtype=np.float32)          # [T-discard, N]
        Y = targets[discard:]                                  # same length

        if self.use_poly:
            feats = np.concatenate(
                [states, states * states, np.ones((states.shape[0], 1), dtype=np.float32)],
                axis=1,
            )
        else:
            feats = states

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(feats, Y)
        self.W_out = reg.coef_.astype(np.float32)             # d_out × feat_dim

    # ======================== autoregressive roll-out =================== #
    def predict_autoregressive(
        self, initial_input: np.ndarray, n_steps: int
    ) -> np.ndarray:
        if self.W_out is None:
            raise RuntimeError("Call fit_readout() before prediction.")

        d_in = initial_input.shape[0]
        preds = np.empty((n_steps, self.W_out.shape[0]), dtype=np.float32)

        #self.reset_state()
        current_in = initial_input.astype(np.float32).copy()

        for t in range(n_steps):
            self._update_state(current_in)

            if self.use_poly:
                big_x = np.concatenate(
                    [self.x, self.x * self.x, np.ones(1, dtype=np.float32)]
                )
            else:
                big_x = self.x

            y_t = (self.W_out @ big_x).astype(np.float32)
            preds[t] = y_t
            current_in = y_t[:d_in]  # feedback: assume d_in ≤ d_out

        return preds
    
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update_state(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


### NRMSE

In [4]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons for Teacher-forced Single-step Forecasting
    """
    horizon_nrmse = {}
    
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets)**2
        variance = np.var(targets, axis=0)
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * variance))
        horizon_nrmse[horizon] = nrmse

    return horizon_nrmse

### VPT

In [5]:
def compute_valid_prediction_time(y_true, y_pred, test_time, lyapunov_time, threshold=0.4):
    y_mean = np.mean(y_true, axis=0)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))

    error = y_true - y_pred
    squared_error = np.sum(error**2, axis=1)
    delta = squared_error / denom

    idx_exceed = np.where(delta > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = test_time[-1]
    else:
        T_VPT = test_time[idx_exceed[0]]

    ratio = T_VPT / lyapunov_time

    return T_VPT, ratio

### ADev

In [6]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

### PSD

In [7]:
def compute_psd(y, dt=0.01):
    z = y[:, 2]  # Extract Z-component
    
    # Compute PSD using Welch’s method
    freqs, psd = welch(z, fs=1/dt, window='hamming', nperseg=len(z))  # Using Hamming window
    
    return freqs, psd

# MIT-BIH Dataset

In [8]:
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb

In [9]:
import wfdb

# Download and load record and annotations for patient #100
record = wfdb.rdrecord('100', sampfrom=0, sampto=25002, pn_dir='mitdb')  # first 20,000 samples
annotation = wfdb.rdann('100', 'atr', sampfrom=0, sampto=25002, pn_dir='mitdb')

In [10]:
# Get input signal u(t) from the first channel
u = record.p_signal[:, 0] 
u

array([-0.145, -0.145, -0.145, ..., -0.41 , -0.415, -0.425],
      shape=(25002,))

In [11]:
# Normalize input
u_min = np.min(u)
u_max = np.max(u)
u_norm = (u - u_min) / (u_max - u_min)

In [12]:
fs = record.fs  # sampling frequency (should be 360 Hz)
t_vals = np.arange(len(u_norm)) / fs

In [13]:
emb_dim = 3
# inputs = u_norm
inputs = create_delay_embedding(u_norm, emb_dim)

# Create target array (heartbeat locations)
targets = np.zeros(len(u_norm))
targets[annotation.sample] = 1  # mark annotations as 1 (heartbeat)
targets = create_delay_embedding(targets, emb_dim)

In [14]:
data_size = len(inputs)
train_size = 15000
train_input = inputs[:train_size]
train_target = targets[:train_size]
test_input = inputs[train_size+1:]
test_target = targets[train_size+1:]
test_size = len(test_input)
print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

Total samples: 25000, train size: 15000, test size: 9999


In [15]:
# result = []
# step = 5
# for a in range(5, 101, step):
#     for b in range(a + step, 111, step):
#         for c in range(b + step, 121, step):
#             for d in range(c + step, 131, step):
#                 for e in range(d + step, 141, step):
#                     for f in range(e + step, 151, step):
#                         for g in range(f + step, 161, step):
#                             for h in range(g + step, 171, step):
#                                 i = 300 - a - b - c - d - e - f - g - h
#                                 if i > h and i % step == 0 and i <= 300:
#                                     result.append([a, b, c, d, e, f, g, h, i])

# print(result)
# print(f"Total unique combinations: {len(result)}")

In [16]:
# grid = {
#     "cells_per_level": [[5, 10, 20, 25, 30, 35, 40, 60, 75, 90, 110]],
#     "spectral_radius": [0.92],
#     "input_scale": [0.1],
#     "leaking_rate": [0.1],
#     "ridge_alpha": [1e-8],
# }

In [17]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    sph_res = MPPRN(
        reservoir_size        = 300,      # total neurons  (must be multiple of 3)
        cycle_weight          = 0.001,      # r  weight on each edge of the three base cycles
        bridge_weight         = 0.02,     # s  weight on the three inter-cycle “bridge” edges
        input_scale           = 1.4,      # |W_in| multiplier   (was 0.5 by default)
        leak_rate             = 0.8,      # α  leaky-integrator coefficient (1.0 ⇒ no leak)
        ridge_alpha           = 1e-6,     #  penalty for the read-out ridge regression
        swirl_beta            = 12.5,      # β  controls the steepness of the swirl gate
        swirl_frequency       = None,     # ω  spatial frequency (defaults to 2π / (N/3))
        swirl_sigmoid         = True,     # if False, uses raw sin; if True, uses σ(β sin(…))
        seed                  = seed,       # seed 
        use_polynomial_readout= True      # augment read-out with square andd bias
    )
    sph_res.fit_readout(train_input, train_target, discard=5000)
    hfr_preds = sph_res.predict_open_loop(test_input)
    hfr_nrmse = evaluate_nrmse(hfr_preds, test_target, horizons)
    nrmse_dict['HFR'].append(hfr_nrmse)

In [18]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'MPPRN':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['HFR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
MPPRN            
--------------------------------------------------------------------------------------------------------------------------------------------
300        0.7022629698063398 ± 0.014567439239971258
600        0.7459315376743182 ± 0.013617995792092273
1000       0.8978069973191508 ± 0.011526372960124492


In [19]:
# def run_grid_search(model_class, param_grid, model_name,
#                     output_path="grid_search_results.json"):
#     # Precompute param combinations
#     combos = list(itertools.product(*param_grid.values()))
#     param_keys = list(param_grid.keys())
#     print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

#     results = []
#     horizons = [300, 600, 1000]
#     seeds = range(995, 1025)
#     # tqdm adds a progress bar for better visualization
#     for comb in tqdm(combos, desc="Grid Search"):
#         params = dict(zip(param_keys, comb))
#         seed_scores = []
#         for seed in seeds:
#             model = model_class(seed=seed, **params)
#             model.fit_readout(train_input, train_target, discard=5000)
#             preds = model.predict_open_loop(test_input)
#             nrmse = evaluate_nrmse(preds, test_target, horizons)
#             seed_scores.append(nrmse)

#         results.append({
#             "params": params,
#             "scores": seed_scores
#         })

#     return results

In [20]:
# results = run_grid_search(
#     HFRRes3D,
#     grid,
#     model_name="HFRRes3D",
#     output_path="hfr_grid_search_results.json"
# )

In [21]:
# results

In [22]:
# import pandas as pd

In [23]:

# rows = []
# for entry in results:
#     params = entry["params"]
#     scores = entry["scores"]

#     # Convert list of dicts to dict of lists per horizon
#     all_scores = {300: [], 600: [], 1000: []}
#     for score_dict in scores:
#         for h in all_scores:
#             all_scores[h].append(score_dict[h])

#     # Compute mean and std for each horizon
#     row = params.copy()
#     for h in all_scores:
#         values = np.array(all_scores[h])
#         row[f"nrmse_{h}_mean"] = values.mean()
#         row[f"nrmse_{h}_std"] = values.std()

#     rows.append(row)

# # Convert to DataFrame
# df = pd.DataFrame(rows)

# # Optional: Convert list-type params to string if needed
# if 'cells_per_level' in df.columns:
#     df['cells_per_level'] = df['cells_per_level'].apply(str)

# # Save to CSV
# df.to_csv("results.csv", index=False)
# print("Saved to grid_search_results.csv")

# Sunspot Dataset

In [24]:
import pandas as pd
file_path = 'datasets/SN_m_tot_V2.0.csv'

df = pd.read_csv(file_path, sep=';', header = None)
df

,0,1,2,3,4,5,6
0,1749,1,1749.042,96.7,-1.0,-1,1
1,1749,2,1749.123,104.3,-1.0,-1,1
2,1749,3,1749.204,116.7,-1.0,-1,1
3,1749,4,1749.288,92.8,-1.0,-1,1
4,1749,5,1749.371,141.7,-1.0,-1,1
...,...,...,...,...,...,...,...
3310,2024,11,2024.873,152.5,20.9,681,0
3311,2024,12,2024.958,154.5,25.6,572,0
3312,2025,1,2025.042,137.0,23.3,670,0
3313,2025,2,2025.122,154.6,23.3,655,0


In [25]:
data = df.iloc[:, 3].values
dt = 1
dataset_size = len(data)
data = create_delay_embedding(data, 3)
print(f"Dataset size: {dataset_size}")

# Train/Test Split
train_end = 2000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}\nTest size: {len(test_input)}")

Dataset size: 3315
Train size: 2000
Test size: 1312


In [ ]:
# all_horizons = list(range(10, 1001, 10))
# horizons = [300, 600, 1000]


# nrmse_dict = defaultdict(list)
# seeds = range(995, 1025)

# for seed in seeds:
#     sph_res = MPPRN(
#         reservoir_size        = 500,      # total neurons  (must be multiple of 3)
#         n_cycles              = 5,
#         cycle_weight          = 0.00001,      # r  weight on each edge of the three base cycles
#         bridge_weight         = 0.02,     # s  weight on the three inter-cycle “bridge” edges
#         input_scale           = 1.1,     # |W_in| multiplier   (was 0.5 by default)
#         leak_rate             = 0.5,      # α  leaky-integrator coefficient (1.0 ⇒ no leak)
#         ridge_alpha           = 1e-8,     #  penalty for the read-out ridge regression
#         swirl_beta            = 150,    # β  controls the steepness of the swirl gate
#         swirl_frequency       = None,     # ω  spatial frequency (defaults to 2π / (N/3))
#         swirl_sigmoid         = True,     # if False, uses raw sin; if True, uses σ(β sin(…))
#         seed                  = seed,       # seed 
#         use_polynomial_readout= True      # augment read-out with square andd bias
#     )
#     sph_res.fit_readout(train_input, train_target, discard=100)
#     hfr_preds = sph_res.predict_open_loop(test_input)
#     hfr_nrmse = evaluate_nrmse(hfr_preds, test_target, horizons)
#     nrmse_dict['HFR'].append(hfr_nrmse)

In [46]:
all_horizons = list(range(10, 1001, 10))
horizons = [300, 600, 1000]


nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    sph_res = MPPRN(
        reservoir_size        = 300,      # total neurons  (must be multiple of 3)
        n_cycles              = 5,
        cycle_weight          = 0.001,      # r  weight on each edge of the three base cycles
        bridge_weight         = 0.02,     # s  weight on the three inter-cycle “bridge” edges
        input_scale           = 1.1,     # |W_in| multiplier   (was 0.5 by default)
        leak_rate             = 0.5,      # α  leaky-integrator coefficient (1.0 ⇒ no leak)
        ridge_alpha           = 1e-8,     #  penalty for the read-out ridge regression
        swirl_beta            = 30,    # β  controls the steepness of the swirl gate
        swirl_frequency       = None,     # ω  spatial frequency (defaults to 2π / (N/3))
        swirl_sigmoid         = True,     # if False, uses raw sin; if True, uses σ(β sin(…))
        seed                  = seed,       # seed 
        use_polynomial_readout= True      # augment read-out with square andd bias
    )
    sph_res.fit_readout(train_input, train_target, discard=100)
    hfr_preds = sph_res.predict_open_loop(test_input)
    hfr_nrmse = evaluate_nrmse(hfr_preds, test_target, horizons)
    nrmse_dict['HFR'].append(hfr_nrmse)

In [47]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'MPPRN':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['HFR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
MPPRN            
--------------------------------------------------------------------------------------------------------------------------------------------
300        0.4655008525792734 ± 0.008749936126559478
600        0.4143967930574644 ± 0.01350331943196619
1000       0.39965414442413544 ± 0.009597118893105184


# Sante Fe Dataset

In [48]:
import pandas as pd

file_path = 'datasets/santa-fe-time-series-competition-data-set-b-1.0.0/b1.txt'

df = pd.read_csv(file_path, header=None, sep=' ')
df

,0,1,2,3
0,76.53,8320,7771,NaN
1,76.53,8117,7774,NaN
2,76.15,7620,7788,NaN
3,75.39,6413,7787,NaN
4,75.51,7518,7767,NaN
...,...,...,...,...
16995,73.57,16021,6498,NaN
16996,73.79,-6957,6547,NaN
16997,74.54,11476,6576,NaN
16998,74.36,15058,6573,NaN


In [49]:
# Normalize the first column (column 0) of the DataFrame
df[0] = (df[0] - df[0].min()) / (df[0].max() - df[0].min())

In [50]:
data = df.iloc[:, 0].values
chosen_system = "SantaFe"
dt = 1
T_data = len(data)
data = create_delay_embedding(data, 3)
print(f"Data length: {T_data}.")

# Train/Test Split
train_end = 7000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}  \nTest size: {len(test_input)}")


Data length: 17000.
Train size: 7000  
Test size: 9997


In [ ]:
all_horizons = list(range(10, 1001, 10))
horizons = [300, 600, 1000]


nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    sph_res = MPPRN(
        reservoir_size        = 300,      # total neurons  (must be multiple of 3)
        n_cycles              = 5,
        cycle_weight          = 0.001,      # r  weight on each edge of the three base cycles
        bridge_weight         = 0.02,     # s  weight on the three inter-cycle “bridge” edges
        input_scale           = 1.1,     # |W_in| multiplier   (was 0.5 by default)
        leak_rate             = 0.5,      # α  leaky-integrator coefficient (1.0 ⇒ no leak)
        ridge_alpha           = 1e-8,     #  penalty for the read-out ridge regression
        swirl_beta            = 50,    # β  controls the steepness of the swirl gate
        swirl_frequency       = None,     # ω  spatial frequency (defaults to 2π / (N/3))
        swirl_sigmoid         = True,     # if False, uses raw sin; if True, uses σ(β sin(…))
        seed                  = seed,       # seed 
        use_polynomial_readout= True      # augment read-out with square andd bias
    )
    sph_res.fit_readout(train_input, train_target, discard=100)
    hfr_preds = sph_res.predict_open_loop(test_input)
    hfr_nrmse = evaluate_nrmse(hfr_preds, test_target, horizons)
    nrmse_dict['HFR'].append(hfr_nrmse)

In [55]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'MPPR-N':<17}")
print("-" * 140)


for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['HFR']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
MPPR-N           
--------------------------------------------------------------------------------------------------------------------------------------------
300        0.2493345548207022 ± 0.0007499412002111277
600        0.2193263750740337 ± 0.0004027635001078921
1000       0.23542138838365562 ± 0.0003896402061993685
